# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [1]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [3]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [4]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [5]:
# ваш код здесь
data.describe(include=object)

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [6]:
# ваш код здесь
df = data.drop(['Education', 'Occupation', 'Commute Distance', 'Region', 'ID'  ], axis=1)
df.head()

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,Married,Female,40000,1,Yes,0,42,No
1,Married,Male,30000,3,Yes,1,43,No
2,Married,Male,80000,5,No,2,60,No
3,Single,Male,70000,0,Yes,1,41,Yes
4,Single,Male,30000,0,No,0,36,Yes


In [7]:
df['Marital Status'].value_counts()

,count
Marital Status,
Married,539
Single,461


In [8]:
# закодируем в 1 и 0
df['Marital Status'] = df['Marital Status'].map({'Married':1, 'Single':0})

In [9]:
df['Gender'].value_counts()

,count
Gender,
Male,509
Female,491


Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [10]:
# ваш код здесь
df['Gender'] = df['Gender'].map({'Male':1, 'Female':0})

In [11]:
df['Home Owner'].value_counts()

,count
Home Owner,
Yes,685
No,315


In [12]:
df['Home Owner'] = df['Home Owner'].map({'Yes':1, 'No':0})


In [13]:
df['Purchased Bike'].value_counts()

,count
Purchased Bike,
No,519
Yes,481


Удалите столбец `ID`, так как он по сути нечисловой.

In [14]:
df['Purchased Bike'] = df['Purchased Bike'].map({'Yes':1, 'No':0})

In [15]:
df.head()

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,1,0,40000,1,1,0,42,0
1,1,1,30000,3,1,1,43,0
2,1,1,80000,5,0,2,60,0
3,0,1,70000,0,1,1,41,1
4,0,1,30000,0,0,0,36,1


In [16]:
# ваш код здесь

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [17]:
# ваш код здесь
X = df.drop('Purchased Bike', axis=1)
y = df['Purchased Bike']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [18]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42) # ваш код здесь

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [19]:
# ваш код здесь
Xtrain.shape

(700, 7)

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [20]:
from sklearn.preprocessing import MinMaxScaler

# ваш код здесь

mms = MinMaxScaler()
Xtrain = pd.DataFrame(mms.fit_transform(Xtrain))
Xtest = pd.DataFrame(mms.transform(Xtest))



Теперь обучите логистическую регрессию на тренировочных данных

In [21]:
from sklearn.linear_model import LogisticRegression

# ваш код здесь

# Создание модели
model = LogisticRegression()
model.fit(Xtrain, ytrain)


LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [22]:
# ваш код здесь train
Y_pred_train = model.predict(Xtrain)
# Предсказание
Y_pred_test = model.predict(Xtest)

Оцените значение accuracy на трейне и на тесте.

In [23]:
from sklearn.metrics import accuracy_score

# ваш код здесь
print("accuracy_score  тестовых данных",accuracy_score(ytest ,Y_pred_test ))
print("accuracy_score на трейне",accuracy_score(ytrain ,Y_pred_train ))

accuracy_score  тестовых данных 0.58
accuracy_score на трейне 0.6342857142857142


Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [24]:
from sklearn.preprocessing import PolynomialFeatures

# ваш код здесь
# Создаем объект PolynomialFeatures, указав степень взаимодействия, равную 2
poly_transformer = PolynomialFeatures(degree = 2)

poly_transformer.fit(Xtrain)

Xtrain = poly_transformer.transform(Xtrain)
Xtest = poly_transformer.transform(Xtest)

print('Polynomial Features shape: ', Xtrain.shape)

Polynomial Features shape:  (700, 36)


**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [25]:
# ваш код здесь
# Создание модели
model = LogisticRegression()
model.fit(Xtrain, ytrain)

# ваш код здесь train
Y_pred_train_poly = model.predict(Xtrain)
# Предсказание
Y_pred_test_poly = model.predict(Xtest)




In [26]:
print("accuracy_score  poly тестовых данных",accuracy_score(ytest , Y_pred_test_poly))
print("accuracy_score на poly трейне",accuracy_score(ytrain , Y_pred_train_poly))

accuracy_score  poly тестовых данных 0.6233333333333333
accuracy_score на poly трейне 0.6871428571428572


**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [27]:
accuracy_score(ytest, model.predict(Xtest))

0.6233333333333333

In [28]:
from sklearn.metrics import recall_score

# ваш код здесь

recall_score(ytest, model.predict(Xtest))

0.5526315789473685

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [29]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 36), (210, 36))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [34]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1

thresholds = np.arange(0, 1, 0.01)
accuracies = []
recalls = []

model.fit(XtrainS, ytrainS)
pred_proba = model.predict_proba(Xval)

for thr in thresholds:
    classes = (pred_proba[:, 1] >= thr)*1
    recall = recall_score(yval, classes)
    acc = accuracy_score(yval, classes)
    accuracies.append(acc)
    recalls.append(recall)

    if recall > RecMax and acc>=0.6:
      RecMax = recall
      BestThr = thr
      BestAcc = acc

print (BestThr, RecMax, BestAcc)

0.32 0.8725490196078431 0.6095238095238096


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [31]:
# ваш код здесь
from sklearn.linear_model import LogisticRegression

# ваш код здесь

# Создание модели
lr = LogisticRegression()
lr.fit(Xtrain, ytrain)

LogisticRegression()

In [33]:
pred_test = lr.predict_proba(Xtest)
pred_test[:10]

array([[0.49619264, 0.50380736],
       [0.55552635, 0.44447365],
       [0.46883522, 0.53116478],
       [0.3131033 , 0.6868967 ],
       [0.71064343, 0.28935657],
       [0.74848493, 0.25151507],
       [0.47314906, 0.52685094],
       [0.56283622, 0.43716378],
       [0.73694565, 0.26305435],
       [0.55011864, 0.44988136]])

In [ ]:
# правильный ответ 0,8, несмотря на полученный 0,9. Фиг его знает почему. Хотя понятно...
# до этого шага была логика, ... а потом "Появились новые требования от заказчика!"

**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.